In [1]:
import sys
sys.path.append('/Users/liuqianyi/Documents/Customer Segmentation Clustering Based on Image Representation for Energy Consumption')
# from preprocess.covert_csv_to_h5 import*

# csv_file = './dataset/15minute_data_newyork.csv'
# metadata = './dataset/metadata.csv' 
# convert_dataport([csv_file], metadata, "15min_newyork.h5", 3e6)

### 1.  Generate daily time series data for each house

In [2]:
from preprocess.generate_time_series import generate_all_1day_15min_list
house_list = generate_all_1day_15min_list('15min_newyork.h5')

In [3]:
print(house_list[0][0])

local_15min
2019-05-01 00:00:00-05:00     994.0
2019-05-01 00:15:00-05:00     747.0
2019-05-01 00:30:00-05:00     605.0
2019-05-01 00:45:00-05:00    1054.0
2019-05-01 01:00:00-05:00     515.0
                              ...  
2019-05-01 22:45:00-05:00     391.0
2019-05-01 23:00:00-05:00     353.0
2019-05-01 23:15:00-05:00     216.0
2019-05-01 23:30:00-05:00     602.0
2019-05-01 23:45:00-05:00     301.0
Freq: 15T, Name: (power, active), Length: 96, dtype: float64


### 2.  Transform to heatmap

In [4]:
from create_heatmaps import*

create_heatmap(house_list)

/Users/liuqianyi/opt/anaconda3/lib/python3.9/site-packages/seaborn/matrix.py:198: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
/Users/liuqianyi/opt/anaconda3/lib/python3.9/site-packages/seaborn/matrix.py:203: RuntimeWarning: All-NaN slice encountered
  vmax = np.nanmax(calc_data)


### 3.  Enhance heatmaps

In [5]:
from enhance_heatmaps import*
image_dir = './15_building_184days_plots'
x,df = enhance_heatmaps(image_dir)

2023-08-30 22:08:47.480867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(2760, 112, 112, 3)


### 4.  Input into CAE model

In [6]:
from CAE_model import *

save_dir = './model_saver'
autoencoder, encoder = CNN_autoencoder(img_shape = (112, 112, 3))
# pretain_autoencoder = pretrain_model(x,autoencoder,save_dir)

In [7]:
autoencoder.input

<KerasTensor: shape=(None, 112, 112, 3) dtype=float32 (created by layer 'input_1')>

### 5.  Evaluation CAE model

In [8]:
from evaluation import *
CAE_evaluation(x,autoencoder)

87/87 [==============================] - 10s 104ms/step
Average RMSE: 0.04389308302883385
Average MAE): 0.024835846908904072


### 6.  Clustering Dimension Reduction Image Data

In [9]:
from proper_k import *
reduction_x = d_x(autoencoder,encoder, x)
k_values = list(range(3, 20))

87/87 [==============================] - 2s 16ms/step
(2760, 265)


### 7.  Clustering Result

In [10]:
from clustering import *
image_folder = './15_building_184days_plots'
if_show_images = 'No'
new_df, select_cluster = kmean_clustering(15, df, reduction_x , image_folder, 3, if_show_images)

/Users/liuqianyi/opt/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [11]:
cluster_check_avg(new_df,13)

                             filename  cluster_label  building_id  day  month  \
3        building_1_day_4_heatmap.png           13.0            1    4      5   
4        building_1_day_5_heatmap.png           13.0            1    5      5   
65      building_1_day_66_heatmap.png           13.0            1   66      7   
70      building_1_day_71_heatmap.png           13.0            1   71      7   
77      building_1_day_78_heatmap.png           13.0            1   78      7   
...                               ...            ...          ...  ...    ...   
2681  building_15_day_106_heatmap.png           13.0           15  106      8   
2743  building_15_day_168_heatmap.png           13.0           15  168     10   
2752  building_15_day_177_heatmap.png           13.0           15  177     10   
2755  building_15_day_180_heatmap.png           13.0           15  180     10   
2756  building_15_day_181_heatmap.png           13.0           15  181     10   

      season  
3          0